In [ ]:
import nipype.interfaces.io as nio  # Data i/o
import nipype.interfaces.fsl as fsl  # fsl
import nipype.interfaces.utility as util  # utility
import nipype.pipeline.engine as pe # pipeline engine
from glob import glob
from os.path import join as oj
import os

In [2]:
FoxDataBaseDir = "/home/dgutman/POST_MORTEM_FOXES/POST_MORTEM_FOXES/"

subject_list = [x for x in os.listdir(FoxDataBaseDir) if x.startswith("Fox_")]
print len(subject_list),"Directories with Fox_ detected"

## These are Erin Generated masks for white and gray matter targets in common space
seedMask =   '/home/dgutman/POST_MORTEM_FOXES/POST_MORTEM_FOXES/Probtrack_Targets/All_Foxes_FA_to_Unselected-Template_GM.nii.gz'
targetMask = '/home/dgutman/POST_MORTEM_FOXES/POST_MORTEM_FOXES/Probtrack_Targets/All_Foxes_FA_to_Unselected-Template_WM.nii.gz'

31 Directories with Fox_ detected


In [3]:
"""
Setup for Probtrackx2 Computational Pipeline
"""
infosource = pe.Node(interface=util.IdentityInterface(fields=['subject_id']),  name="infosource")
infosource.iterables = ('subject_id', subject_list[0:2])
### Above just converts the list of subjects into an iterable list I can connect to the next part of the pipeline

info = dict(
             thsamples = [['subject_id','th1samples']],
             phsamples =  [['subject_id','ph1samples']],
             fsamples =  [['subject_id','f1samples']]   ,
             nodif_brain_mask = [['subject_id','nodif_brain_mask']],
             xfm = [['subject_id','Unselected-Template_FNIRT-warpfield_T2.nii.gz']],
             invxfm = [['subject_id', 'T2_FNIRT-warpfield_Unselected-Template.nii.gz' ]]
            
)
### Generate the files/names for the DTI/bedpostX data
dti_datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'], outfields=info.keys() ),   name='dti_datasource')
dti_datasource.inputs.base_directory = FoxDataBaseDir
dti_datasource.inputs.template = "*"
dti_datasource.inputs.sort_filelist=True
dti_datasource.inputs.field_template = dict( 
    nodif_brain_mask = '%s/data.bedpostX/%s.nii.gz',    
    thsamples='%s/data.bedpostX/merged_%s.nii.gz',
    phsamples='%s/data.bedpostX/merged_%s.nii.gz',
    fsamples='%s/data.bedpostX/merged_%s.nii.gz',
    xfm = "%s/xfms/%s",
    invxfm = "%s/xfms/%s"
)
dti_datasource.inputs.template_args = info

In [1]:
pbx2 = pe.Node(interface=fsl.ProbTrackX2(), name='pbx2')
# probtrackx.inputs.mode = 'seedmask'
pbx2.inputs.c_thresh = 0.2   # -c 0.2
pbx2.inputs.n_steps = 2000   # -S 2000
pbx2.inputs.step_length = 0.5 # --steplength=0.5
pbx2.inputs.n_samples = 5000 # -P 5000
pbx2.inputs.opd = True
pbx2.inputs.os2t = True
pbx2.inputs.loop_check = True
pbx2.inputs.onewaycondition = True # --onewaycondition
pbx2.inputs.omatrix3 = True   # --omatrix3
pbx2.inputs.target3 = targetMask # --target3=$WORKINGDATAPATH/All_Foxes_FA_to_Unselected-Template_GM.nii.gz "; 
pbx2.inputs.correct_path_distribution = True # -pd
pbx2.inputs.seed = seedMask

pbx2.inputs.samples_base_name = 'merged'

pbx2.inputs.lrtarget3= targetMask  ## This is a hack for now

# # Run tractography with warp to/from template space
# $statement .= " -s $WORKINGDATAPATH/Fox_" . $subj[$k] . "/data.bedpostX/merged "; 
# $statement .= " -x $WORKINGDATAPATH/All_Foxes_FA_to_Unselected-Template_WM.nii.gz ";  ## check downsample
# $statement .= " -l   --fibthresh=0.1 --randfib=0 ";  ## check # of samples
# $statement .= " --forcedir --opd ";
# $statement .= " --dir=$WORKINGDATAPATH/Fox_" . $subj[$k] . "/data.bedpostX/WholeBrainMatrixConnectivity_TemplateSpace_fullres/ ";
# print "$statement \n";

NameError: name 'pe' is not defined

In [ ]:
runpbx2  = pe.Workflow(name="runpbx2")
runpbx2.base_dir = "/NIPYPE_SCRATCH/"

runpbx2.connect(infosource,'subject_id',dti_datasource,'subject_id')


# samples_base_name_fxn = lambda x : x[0].replace('_th1samples.nii.gz','')


# ### Connect the dti_datasource to the pbx2 command
runpbx2.connect( dti_datasource,'thsamples',pbx2,'thsamples')
runpbx2.connect( dti_datasource,'phsamples',pbx2,'phsamples')
runpbx2.connect( dti_datasource,'fsamples',pbx2,'fsamples')
runpbx2.connect( dti_datasource,'nodif_brain_mask',pbx2,'mask')
# runpbx2.connect( dti_datasource, ('thsamples',samples_base_name_fxn) ,pbx2,'samples_base_name')

In [11]:
runpbx2.run()

180426-17:48:04,183 workflow INFO:
	 Workflow runpbx2 settings: ['check', 'execution', 'logging', 'monitoring']
180426-17:48:04,209 workflow INFO:
	 Running serially.
180426-17:48:04,210 workflow INFO:
	 [Node] Setting-up "runpbx2.dti_datasource" in "/NIPYPE_SCRATCH/runpbx2/_subject_id_Fox_10T/dti_datasource".
180426-17:48:04,232 workflow INFO:
	 [Node] Running "dti_datasource" ("nipype.interfaces.io.DataGrabber")
180426-17:48:04,250 workflow INFO:
	 [Node] Finished "runpbx2.dti_datasource".
180426-17:48:04,252 workflow INFO:
	 [Node] Setting-up "runpbx2.pbx2" in "/NIPYPE_SCRATCH/runpbx2/_subject_id_Fox_10T/pbx2".
180426-17:48:04,290 workflow INFO:
	 [Node] Running "pbx2" ("nipype.interfaces.fsl.dti.ProbTrackX2"), a CommandLine Interface with command:
probtrackx2 --cthr=0.200 --pd --forcedir --loopcheck --lrtarget3=/home/dgutman/POST_MORTEM_FOXES/POST_MORTEM_FOXES/Probtrack_Targets/All_Foxes_FA_to_Unselected-Template_WM.nii.gz -m /home/dgutman/POST_MORTEM_FOXES/POST_MORTEM_FOXES/Fox_10

RuntimeError: Workflow did not execute cleanly. Check log for details

In [ ]:



# from nipype.interfaces import fsl
# pbx2 = fsl.ProbTrackX2()
#  pbx2.inputs.seed = 'seed_source.nii.gz'
# >>> pbx2.inputs.thsamples = 'merged_th1samples.nii.gz'
# >>> pbx2.inputs.fsamples = 'merged_f1samples.nii.gz'
# >>> pbx2.inputs.phsamples = 'merged_ph1samples.nii.gz'
# >>> pbx2.inputs.mask = 'nodif_brain_mask.nii.gz'
# >>> pbx2.inputs.out_dir = '.'
# >>> pbx2.inputs.n_samples = 3
# >>> pbx2.inputs.n_steps = 10
# >>> pbx2.cmdline
# 'probtrackx2 --forcedir -m nodif_brain_mask.nii.gz --nsamples=3 --nsteps=10 --opd --dir=. --samples=merged --seed=seed_source.nii.gz'

In [ ]:

# # runpbx2.connect(
# #      [(infosource, datasource, [('subject_id', 'subject_id')]),
    
    
    
# #     [(bedpostx, probtrackx,
# #                        [('outputnode.thsamples',
# #                          'thsamples'), ('outputnode.phsamples', 'phsamples'),
# #                         ('outputnode.fsamples', 'fsamples')]),
# #                       (probtrackx, findthebiggest, [('targets', 'in_files')]),
# #                       (flirt, probtrackx, [('out_matrix_file', 'xfm')])])
# """
# Setup data storage area
# """

# datasink = pe.Node(interface=nio.DataSink(), name='datasink')
# datasink.inputs.base_directory = os.path.abspath('dtiresults')


# dwiproc.base_dir = os.path.abspath('fsl_dti_tutorial')
# dwiproc.connect(
#     [(infosource, datasource, [('subject_id', 'subject_id')]),
#      (datasource, computeTensor,
#       [('dwi', 'fslroi.in_file'), ('bvals', 'dtifit.bvals'),
#        ('bvecs', 'dtifit.bvecs'), ('dwi', 'eddycorrect.inputnode.in_file')]),
#      (datasource, tractography,
#       [('bvals', 'bedpostx.inputnode.bvals'),
#        ('bvecs', 'bedpostx.inputnode.bvecs'), ('seed_file', 'probtrackx.seed'),
#        ('target_masks', 'probtrackx.target_masks')]),
#      (computeTensor, tractography,
#       [('eddycorrect.outputnode.eddy_corrected', 'bedpostx.inputnode.dwi'),
#        ('bet.mask_file', 'bedpostx.inputnode.mask'), ('bet.mask_file',
#                                                       'probtrackx.mask'),
#        ('fslroi.roi_file', 'flirt.reference')]), (infosource, datasink, [
#            ('subject_id', 'container'), (('subject_id', getstripdir),
#                                          'strip_dir')
#        ]), (tractography, datasink, [('findthebiggest.out_file',
#                                       'fbiggest.@biggestsegmentation')])])


# stout_tractography.connect( dti_datasource, 'nodif_brain_mask', cvtwarp_dti_to_mni, 'reference')
# stout_tractography.connect( dti_datasource, 'MNI_to_struct', cvtwarp_dti_to_mni, 'warp1')
# stout_tractography.connect( dti_datasource, 'struct_to_nodif_aff', cvtwarp_dti_to_mni, 'postmat')

      
      # #PERFORM PROBALISTIC TRACTOGRAPHY

# pbx2 = pe.Node(interface=fsl.ProbTrackX2(),name='pbx2')
# pbx2.inputs.c_thresh = 0.2
# pbx2.inputs.n_steps=2000
# pbx2.inputs.step_length=0.5
# pbx2.inputs.n_samples= 5000 ### Make n_samples an iterable field...
# pbx2.inputs.opd=True
# pbx2.inputs.loop_check=True
# pbx2.inputs.omatrix2=True  ##This requires I include target2
# pbx2.inputs.correct_path_distribution=True ##corresponds to the --pd flag
# pbx2.inputs.onewaycondition=True  
# pbx2.inputs.target2= os.path.join(RAWDATAPATH,"MNI152_T1_1mm_brain_mask_downsample_2.nii.gz")
# pbx2.inputs.rand_fib=0

# ### This will eventually become an iterable
# pbx2.inputs.seed= [os.path.join(RAWDATAPATH,"ROIs/Human_IFG-vPrCG_L.nii.gz"),os.path.join(RAWDATAPATH,"ROIs/Human_IFG-vPrCG_R.nii.gz")]

# ### Now connect the warps generated from the previous steps

# stout_tractography.connect( cvtwarp_dti_to_mni, 'out_file',pbx2,'xfm')
# stout_tractography.connect( cvtwarp_mni_to_dti, 'out_file',pbx2,'inv_xfm')

# # preuss_tractography.connect( roi_datasource, 'seed_rois',probtrackx,'seed')
# #preuss_tractography.connect( seed_mask_ds,'out_files',probtrackx,'seed')
# #preuss_tractography.connect( dti_dti_datasource, ('subject_id',mask_info),probtrackx,'seed')

# """
# Setup data storage area for results
# """
# datasink = pe.Node(interface=nio.DataSink(),name='datasink')

# stout_tractography.connect( pbx2, 'fdt_paths', datasink, 'fdt_paths')
# stout_tractography.connect( pbx2, 'matrix1_dot', datasink, 'matrix1.txt')
# stout_tractography.connect( pbx2, 'matrix2_dot', datasink, 'matrix2.txt')
# stout_tractography.connect( pbx2, 'matrix3_dot', datasink, 'matrix3.txt')
# stout_tractography.connect( pbx2, 'lookup_tractspace', datasink, 'lookup_ts')
# stout_tractography.connect( pbx2, 'way_total', datasink, 'waytotal.txt')


# #datasink.inputs.base_directory = put_cleaned_up_results_here
# #datasink.inputs.substitutions = [('_subject_id_', ''),
# #                                 ('_seed_..bigdata2..NIPYPE_WD..SANCHEZ_WD..', '')]

# if __name__ == '__main__':
#     stout_tractography.write_graph(format='eps')
#     #stout_tractography.export_graph()
#     # stout_tractography.run(plugin='MultiProc', plugin_args={'n_procs': 28})
#     #dwiproc.run(plugin='PBS', plugin_args = dict(qsub_args='-k oe -q batch ') )
#     stout_tractography.run(plugin='PBS', plugin_args=dict(qsub_args='-e trq_logs -o trq_logs -q batch '))

# """


# ###WORKINGDATAPATH

# # $statement  = " convertwarp ";
# # $statement .= " --ref=$WORKINGDATAPATH/MNI152_T1_1mm_brain.nii.gz ";
# # $statement .= " --premat=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc.mat ";
# # $statement .= " --warp1=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_warp_MNI_warpfield.nii.gz ";
# # $statement .= " --out=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc_warp_MNI_warpfield.nii.gz ";
# # print "$statement \n";

# # $statement  = " convertwarp ";
# # $statement .= " --ref=$WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain_mask.nii.gz ";
# # $statement .= " --warp1=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/MNI_warp_struc_warpfield.nii.gz ";
# # $statement .= " --postmat=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_12dof_nodif.mat ";
# # $statement .= " --out=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/MNI_warp_struc_12dof_nodif_warpfield.nii.gz ";
# # print "$statement \n";

# # print "$statement \n";
# """



def getstripdir(subject_id):
    import os
    return os.path.join(
        os.path.abspath('data/workingdir/dwiproc'),
        '_subject_id_%s' % subject_id)

In [ ]:
import nipype.pipelinepeline as pe


# #!/usr/bin/perl

# $WORKINGDATAPATH = "/home/ehecht/POST_MORTEM_FOXES/";

# $subjSTRING="13U 14U 15U 16U 17U 18U 20U 22U 23U 24U 10T 11T 12T 2T 3T 4T 5T 7T 8T 9T 49A 50A 51A 52A 54A 56A 57A 58A 59A 60A";  
# @subj = split(/ /,$subjSTRING);

# $dsSTRING="0.9"; #1.5
# @ds = split(/ /,$dsSTRING);

# $splitSTRING="0-440 441-881 882-1322 1323-1764"; 
# @split = split(/ /,$splitSTRING);

# for($k=0;$k<=$#subj;$k++) 		{
# for($d=0;$d<=$#ds;$d++) 		{

# print "#\$ -cwd\n";
# print "#\$ -S /bin/bash \n";
# print "export FSLDIR=/usr/local/fsl  \n";
# print ". /usr/local/fsl/etc/fslconf/fsl.sh \n";
# print "export PATH=\$PATH:\$FSLDIR/bin \n";
# print "#!/bin/bash \n";

# # Run tractography with warp to/from template space
# $statement  = " probtrackx2 --pd --onewaycondition --omatrix3 ";
# $statement .= " -s $WORKINGDATAPATH/Fox_" . $subj[$k] . "/data.bedpostX/merged "; 
# $statement .= " -m $WORKINGDATAPATH/Fox_" . $subj[$k] . "/data.bedpostX/nodif_brain_mask.nii.gz ";
# $statement .= " -x $WORKINGDATAPATH/All_Foxes_FA_to_Unselected-Template_WM.nii.gz ";  ## check downsample
# $statement .= " --target3=$WORKINGDATAPATH/All_Foxes_FA_to_Unselected-Template_GM.nii.gz ";  ## check downsample
# $statement .= " --xfm=$WORKINGDATAPATH/Fox_" . $subj[$k] . "/xfms/Unselected-Template_FNIRT-warpfield_T2.nii.gz ";
# $statement .= " --invxfm=$WORKINGDATAPATH/Fox_" . $subj[$k] . "/xfms/T2_FNIRT-warpfield_Unselected-Template.nii.gz ";
# $statement .= " -l -c 0.2 -S 2000 --steplength=0.5 -P 5000 --fibthresh=0.1 --randfib=0 ";  ## check # of samples
# $statement .= " --forcedir --opd ";
# $statement .= " --dir=$WORKINGDATAPATH/Fox_" . $subj[$k] . "/data.bedpostX/WholeBrainMatrixConnectivity_TemplateSpace_fullres/ ";
# print "$statement \n";

# }

In [ ]:
# >>> from nipype.interfaces import fsl
# >>> pbx2 = fsl.ProbTrackX2()
# >>> pbx2.inputs.seed = 'seed_source.nii.gz'
# >>> pbx2.inputs.thsamples = 'merged_th1samples.nii.gz'
# >>> pbx2.inputs.fsamples = 'merged_f1samples.nii.gz'
# >>> pbx2.inputs.phsamples = 'merged_ph1samples.nii.gz'
# >>> pbx2.inputs.mask = 'nodif_brain_mask.nii.gz'
# >>> pbx2.inputs.out_dir = '.'
# >>> pbx2.inputs.n_samples = 3
# >>> pbx2.inputs.n_steps = 10
# >>> pbx2.cmdline
# 'probtrackx2 --forcedir -m nodif_brain_mask.nii.gz --nsamples=3 --nsteps=10 --opd --dir=. --samples=merged --seed=seed_source.nii.gz'

In [ ]:
#!/usr/bin/env python
"""
==============
dMRI: DTI, FSL
==============
A pipeline example that uses several interfaces to perform analysis on
diffusion weighted images using FSL FDT tools.
This tutorial is based on the 2010 FSL course and uses data freely available at
the FSL website at: http://www.fmrib.ox.ac.uk/fslcourse/fsl_course_data2.tar.gz
More details can be found at
http://www.fmrib.ox.ac.uk/fslcourse/lectures/practicals/fdt/index.htm
In order to run this tutorial you need to have fsl tools installed and
accessible from matlab/command line. Check by calling fslinfo from the command
line.
Tell python where to find the appropriate functions.
"""

import nipype.interfaces.io as nio  # Data i/o
import nipype.interfaces.fsl as fsl  # fsl
import nipype.interfaces.utility as util  # utility
import nipype.pipeline.engine as pe  # pypeline engine
import os  # system functions
from nipype.workflows.dmri.fsl.dti import create_eddy_correct_pipeline,\
    create_bedpostx_pipeline
"""
Confirm package dependencies are installed.  (This is only for the
tutorial, rarely would you put this in your own code.)
"""

from nipype.utils.misc import package_check

package_check('numpy', '1.3', 'tutorial1')
package_check('scipy', '0.7', 'tutorial1')
package_check('IPython', '0.10', 'tutorial1')
"""
Setting up workflows
--------------------
This is a generic workflow for DTI data analysis using the FSL
Data specific components
------------------------
The nipype tutorial contains data for two subjects.  Subject data is in two
subdirectories, ``dwis1`` and ``dwis2``.  Each subject directory contains each
of the following files: bvec, bval, diffusion weighted data, a set of target
masks, a seed file, and a transformation matrix.
Below we set some variables to inform the ``datasource`` about the
layout of our data.  We specify the location of the data, the subject
sub-directories and a dictionary that maps each run to a mnemonic (or
field) for the run type (``dwi`` or ``bvals``).  These fields become
the output fields of the ``datasource`` node in the pipeline.
Specify the subject directories
"""

subject_list = ['subj1']
"""
Map field names to individual subject runs
"""

info = dict(
    dwi=[['subject_id', 'data']],
    bvecs=[['subject_id', 'bvecs']],
    bvals=[['subject_id', 'bvals']],
    seed_file=[['subject_id', 'MASK_average_thal_right']],
    target_masks=[[
        'subject_id', [
            'MASK_average_M1_right', 'MASK_average_S1_right',
            'MASK_average_occipital_right', 'MASK_average_pfc_right',
            'MASK_average_pmc_right', 'MASK_average_ppc_right',
            'MASK_average_temporal_right'
        ]
    ]])

infosource = pe.Node(
    interface=util.IdentityInterface(fields=['subject_id']), name="infosource")
"""
Here we set up iteration over all the subjects. The following line
is a particular example of the flexibility of the system.  The
``datasource`` attribute ``iterables`` tells the pipeline engine that
it should repeat the analysis on each of the items in the
``subject_list``. In the current example, the entire first level
preprocessing and estimation will be repeated for each subject
contained in subject_list.
"""

infosource.iterables = ('subject_id', subject_list)
"""
Now we create a :class:`nipype.interfaces.io.DataGrabber` object and
fill in the information from above about the layout of our data.  The
:class:`nipype.pipeline.engine.Node` module wraps the interface object
and provides additional housekeeping and pipeline specific
functionality.
"""

datasource = pe.Node(
    interface=nio.DataGrabber(
        infields=['subject_id'], outfields=list(info.keys())),
    name='datasource')

datasource.inputs.template = "%s/%s"

# This needs to point to the fdt folder you can find after extracting
# http://www.fmrib.ox.ac.uk/fslcourse/fsl_course_data2.tar.gz
datasource.inputs.base_directory = os.path.abspath('fsl_course_data/fdt/')

datasource.inputs.field_template = dict(
    dwi='%s/%s.nii.gz',
    seed_file="%s.bedpostX/%s.nii.gz",
    target_masks="%s.bedpostX/%s.nii.gz")
datasource.inputs.template_args = info
datasource.inputs.sort_filelist = True
"""
Setup for Diffusion Tensor Computation
--------------------------------------
Here we will create a generic workflow for DTI computation
"""

computeTensor = pe.Workflow(name='computeTensor')
"""
extract the volume with b=0 (nodif_brain)
"""

fslroi = pe.Node(interface=fsl.ExtractROI(), name='fslroi')
fslroi.inputs.t_min = 0
fslroi.inputs.t_size = 1
"""
create a brain mask from the nodif_brain
"""

bet = pe.Node(interface=fsl.BET(), name='bet')
bet.inputs.mask = True
bet.inputs.frac = 0.34
"""
correct the diffusion weighted images for eddy_currents
"""

eddycorrect = create_eddy_correct_pipeline('eddycorrect')
eddycorrect.inputs.inputnode.ref_num = 0
"""
compute the diffusion tensor in each voxel
"""

dtifit = pe.Node(interface=fsl.DTIFit(), name='dtifit')
"""
connect all the nodes for this workflow
"""

computeTensor.connect(
    [(fslroi, bet, [('roi_file', 'in_file')]),
     (eddycorrect, dtifit, [('outputnode.eddy_corrected', 'dwi')]),
     (infosource, dtifit,
      [['subject_id', 'base_name']]), (bet, dtifit, [('mask_file', 'mask')])])
"""
Setup for Tracktography
-----------------------
Here we will create a workflow to enable probabilistic tracktography
and hard segmentation of the seed region
"""

tractography = pe.Workflow(name='tractography')
tractography.base_dir = os.path.abspath('fsl_dti_tutorial')
"""
estimate the diffusion parameters: phi, theta, and so on
"""

bedpostx = create_bedpostx_pipeline()
bedpostx.get_node("xfibres").iterables = ("n_fibres", [1, 2])

flirt = pe.Node(interface=fsl.FLIRT(), name='flirt')
flirt.inputs.in_file = fsl.Info.standard_image('MNI152_T1_2mm_brain.nii.gz')
flirt.inputs.dof = 12
"""
perform probabilistic tracktography
"""

probtrackx = pe.Node(interface=fsl.ProbTrackX(), name='probtrackx')
probtrackx.inputs.mode = 'seedmask'
probtrackx.inputs.c_thresh = 0.2
probtrackx.inputs.n_steps = 2000
probtrackx.inputs.step_length = 0.5
probtrackx.inputs.n_samples = 5000
probtrackx.inputs.opd = True
probtrackx.inputs.os2t = True
probtrackx.inputs.loop_check = True
"""
perform hard segmentation on the output of probtrackx
"""

findthebiggest = pe.Node(interface=fsl.FindTheBiggest(), name='findthebiggest')
"""
connect all the nodes for this workflow
"""

tractography.add_nodes([bedpostx, flirt])
tractography.connect([(bedpostx, probtrackx,
                       [('outputnode.thsamples',
                         'thsamples'), ('outputnode.phsamples', 'phsamples'),
                        ('outputnode.fsamples', 'fsamples')]),
                      (probtrackx, findthebiggest, [('targets', 'in_files')]),
                      (flirt, probtrackx, [('out_matrix_file', 'xfm')])])
"""
Setup data storage area
"""

datasink = pe.Node(interface=nio.DataSink(), name='datasink')
datasink.inputs.base_directory = os.path.abspath('dtiresults')


def getstripdir(subject_id):
    import os
    return os.path.join(
        os.path.abspath('data/workingdir/dwiproc'),
        '_subject_id_%s' % subject_id)


"""
Setup the pipeline that combines the 2 workflows: tractography & computeTensor
------------------------------------------------------------------------------
"""

dwiproc = pe.Workflow(name="dwiproc")
dwiproc.base_dir = os.path.abspath('fsl_dti_tutorial')
dwiproc.connect(
    [(infosource, datasource, [('subject_id', 'subject_id')]),
     (datasource, computeTensor,
      [('dwi', 'fslroi.in_file'), ('bvals', 'dtifit.bvals'),
       ('bvecs', 'dtifit.bvecs'), ('dwi', 'eddycorrect.inputnode.in_file')]),
     (datasource, tractography,
      [('bvals', 'bedpostx.inputnode.bvals'),
       ('bvecs', 'bedpostx.inputnode.bvecs'), ('seed_file', 'probtrackx.seed'),
       ('target_masks', 'probtrackx.target_masks')]),
     (computeTensor, tractography,
      [('eddycorrect.outputnode.eddy_corrected', 'bedpostx.inputnode.dwi'),
       ('bet.mask_file', 'bedpostx.inputnode.mask'), ('bet.mask_file',
                                                      'probtrackx.mask'),
       ('fslroi.roi_file', 'flirt.reference')]), (infosource, datasink, [
           ('subject_id', 'container'), (('subject_id', getstripdir),
                                         'strip_dir')
       ]), (tractography, datasink, [('findthebiggest.out_file',
                                      'fbiggest.@biggestsegmentation')])])

if __name__ == '__main__':
    dwiproc.run()
    dwiproc.write_graph()

In [ ]:
#!/usr/bin/python

"""
This is the ehecht DATA Set
===============
dMRI [DTI, FSL]
===============
This pipeline example that uses several interfaces to perform analysis on
diffusion weighted images using FSL FDT tools.

I stripped out the bedpostX generation part and modified the dti_dti_datasource
to point directory to a bedpostX directory
"""
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.fsl as fsl          # fsl
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine
import os                                    # system functions
import glob
import sys

fsl.FSLCommand.set_default_output_type('NIFTI_GZ')

RAWDATAPATH='/GLOBAL_SCRATCH/ERIN_TEST_DATA/Stout_Human/'
WORKINGDATAPATH='/GLOBAL_SCRATCH/ERIN_TEST_DATA/STOUT_WD/'

### Could also simply autodetect this from the path...
subject_list = ['Subj01_Scan1','Subj01_Scan2','Subj01_Scan3', 'Subj02_Scan1', 'Subj02_Scan2a', \
'Subj02_Scan2b', 'Subj03_Scan1',  'Subj03_Scan2a', 'Subj03_Scan2b', 'Subj03_Scan3', 'Subj04_Scan1', \
'Subj05_Scan1', 'Subj05_Scan2', 'Subj05_Scan3', 'Subj06_Scan1', 'Subj06_Scan2a',  'Subj06_Scan2b' ]


subj_list = [ x for x in os.listdir(RAWDATAPATH) if 'Subj' in x ]
subject_list = subj_list  ### building a larger subject list

"""
Setting up workflows
--------------------
This is a generic workflow for DTI data analysis using the that I have adapted for  Ehecht Stout
"""
print len(subject_list),"entries to process"

"""
Map field names to individual subject runs
"""

infosource = pe.Node(interface=util.IdentityInterface(fields=['subject_id']),    name="infosource")
infosource.iterables = ('subject_id', subject_list)
### Above just converts the list of subjects into an iterable list I can connect to the next part of the pipeline

info = dict(
             bvecs = [['subject_id','bvecs']],
            bvals = [['subject_id','bvals']],
             thsamples = [['subject_id','th1samples']],
             phsamples =  [['subject_id','ph1samples']],
             fsamples =  [['subject_id','f1samples']]   ,
             nodif_brain_mask = [['subject_id','nodif_brain_mask']],
             nodif_to_struct_aff =   [['subject_id','nodif_12dof_struc.mat']],
	     struct_warpto_MNI = [['subject_id', 'struc_warp_MNI_warpfield.nii.gz']],
	     struct_to_nodif_aff = [['subject_id','struc_12dof_nodif.mat']],
             MNI_to_struct = [['subject_id','MNI_warp_struc_warpfield.nii.gz']]
                                 )

#             seed_region_base_dir = [['subject_id']],
##             xfm_base_dir = [['subject_id']],

### Generate the files/names for the DTI/bedpostX data
dti_datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'], outfields=info.keys() ),   name='dti_datasource')
dti_datasource.inputs.base_directory = RAWDATAPATH
dti_datasource.inputs.template = "*"
dti_datasource.inputs.sort_filelist=True
dti_datasource.inputs.field_template = dict( nodif_brain_mask = '%s/DTI/data/%s.nii.gz',
        bvecs='%s/DTI/data/%s', bvals='%s/DTI/data/%s',  
  thsamples='%s/DTI/data.bedpostX/merged_%s.nii.gz', phsamples='%s/DTI/data.bedpostX/merged_%s.nii.gz',
  fsamples='%s/DTI/data.bedpostX/merged_%s.nii.gz', nodif_to_struct_aff = '%s/xfms/%s', 
		struct_warpto_MNI='%s/xfms/%s', struct_to_nodif_aff='%s/xfms/%s', MNI_to_struct='%s/xfms/%s'
     )
dti_datasource.inputs.template_args = info

# roi_datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'],outfields = ['seed_rois'],  name='roi_datasource')
# roi_datasource.inputs.base_directory = RAWDATAPATH
# roi_datasource.inputs.template = "*"
# roi_datasource.inputs.field_template = {'seed_rois': '%s/ROIs/warped_ROIs_in_cropped_struct_space/Bar*.nii.gz'   }
# roi_datasource.inputs.template_args = {'seed_rois': [['subject_id']] }


"""
Setup for Tracktography
-----------------------
Here we will create a generic workflow for DTI computation, this workflow assumes the .bedpostX has already been run...
Here we will create a workflow to enable probabilistic tracktography and hard segmentation of the seed region
"""
stout_tractography = pe.Workflow(name='stout_tractography')
stout_tractography.base_dir = WORKINGDATAPATH

### First connect the subject list to the workflow to build up the patient lists
stout_tractography.connect( infosource,'subject_id',dti_datasource, 'subject_id' )

### GENERATE THE WARP FIELDS
cvtwarp_mni_to_dti = pe.Node(interface=fsl.ConvertWarp(),name='cvtwarp_mni_to_dti' )
cvtwarp_mni_to_dti.inputs.reference = os.path.join(RAWDATAPATH,"MNI152_T1_1mm_brain.nii.gz")

stout_tractography.connect( dti_datasource, 'nodif_to_struct_aff', cvtwarp_mni_to_dti, 'premat')
stout_tractography.connect( dti_datasource, 'struct_warpto_MNI', cvtwarp_mni_to_dti, 'warp1')

# $statement .= " --premat=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc.mat ";
# $statement .= " --warp1=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_warp_MNI_warpfield.nii.gz ";
# $statement .= " --out=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc_warp_MNI_warpfield.nii.gz ";

cvtwarp_dti_to_mni = pe.Node(interface=fsl.ConvertWarp(),name='cvtwarp_dti_to_mni'  )
stout_tractography.connect( dti_datasource, 'nodif_brain_mask', cvtwarp_dti_to_mni, 'reference')
stout_tractography.connect( dti_datasource, 'MNI_to_struct', cvtwarp_dti_to_mni, 'warp1')
stout_tractography.connect( dti_datasource, 'struct_to_nodif_aff', cvtwarp_dti_to_mni, 'postmat')

# $statement .= " --ref=$WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain_mask.nii.gz ";
# $statement .= " --warp1=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/MNI_warp_struc_warpfield.nii.gz ";
# $statement .= " --postmat=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_12dof_nodif.mat ";
# $statement .= " --out=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/MNI_warp_struc_12dof_nodif_warpfield.nii.gz ";

#PERFORM PROBALISTIC TRACTOGRAPHY

pbx2 = pe.Node(interface=fsl.ProbTrackX2(),name='pbx2')
pbx2.inputs.c_thresh = 0.2
pbx2.inputs.n_steps=2000
pbx2.inputs.step_length=0.5
pbx2.inputs.n_samples= 5000 ### Make n_samples an iterable field...
pbx2.inputs.opd=True
pbx2.inputs.loop_check=True
pbx2.inputs.omatrix2=True  ##This requires I include target2
pbx2.inputs.correct_path_distribution=True ##corresponds to the --pd flag
pbx2.inputs.onewaycondition=True  
pbx2.inputs.target2= os.path.join(RAWDATAPATH,"MNI152_T1_1mm_brain_mask_downsample_2.nii.gz")
pbx2.inputs.rand_fib=0

### This will eventually become an iterable
pbx2.inputs.seed= [os.path.join(RAWDATAPATH,"ROIs/Human_IFG-vPrCG_L.nii.gz"),os.path.join(RAWDATAPATH,"ROIs/Human_IFG-vPrCG_R.nii.gz")]


# roi_list = [x for x in os.path.join(RAWDATAPATH,"ROIs/Human_*/*.nio.gz")]

# $statement  = " probtrackx2 --pd --onewaycondition --omatrix2 ";
# $statement .= " -s $WORKINGDATAPATH/" . $subj[$i] . "/DTI/data.bedpostX/merged "; 
# $statement .= " -m $WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain_mask.nii.gz ";
# $statement .= " -x $WORKINGDATAPATH/ROIs/Human_" . $roi[$r] . "_" . $hemi[$h] . ".nii.gz ";
# $statement .= " --target2=$WORKINGDATAPATH/MNI152_T1_1mm_brain_mask_downsample_2.nii.gz ";
# $statement .= " -l -c 0.2 -S 2000 --steplength=0.5 -P 5000 --fibthresh=0.1 --randfib=0 ";  # check samples 
# $statement .= " --xfm=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/MNI_warp_struc_12dof_nodif_warpfield.nii.gz ";
# $statement .= " --invxfm=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc_warp_MNI_warpfield.nii.gz ";
# $statement .= " --forcedir --opd ";
# $statement .= " --dir=$OUTPUTDATAPATH/" . $subj[$i] . "_" . $roi[$r] . "_" . $hemi[$h] . "_Segmentation  ";

stout_tractography.connect( dti_datasource,'thsamples',pbx2,'thsamples')
stout_tractography.connect( dti_datasource,'phsamples',pbx2,'phsamples')
stout_tractography.connect( dti_datasource,'fsamples',pbx2,'fsamples')
stout_tractography.connect( dti_datasource,'nodif_brain_mask',pbx2,'mask')

### Now connect the warps generated from the previous steps

stout_tractography.connect( cvtwarp_dti_to_mni, 'out_file',pbx2,'xfm')
stout_tractography.connect( cvtwarp_mni_to_dti, 'out_file',pbx2,'inv_xfm')

# preuss_tractography.connect( roi_datasource, 'seed_rois',probtrackx,'seed')
#preuss_tractography.connect( seed_mask_ds,'out_files',probtrackx,'seed')
#preuss_tractography.connect( dti_dti_datasource, ('subject_id',mask_info),probtrackx,'seed')

"""
Setup data storage area for results
"""
datasink = pe.Node(interface=nio.DataSink(),name='datasink')

stout_tractography.connect( pbx2, 'fdt_paths', datasink, 'fdt_paths')
stout_tractography.connect( pbx2, 'matrix1_dot', datasink, 'matrix1.txt')
stout_tractography.connect( pbx2, 'matrix2_dot', datasink, 'matrix2.txt')
stout_tractography.connect( pbx2, 'matrix3_dot', datasink, 'matrix3.txt')
stout_tractography.connect( pbx2, 'lookup_tractspace', datasink, 'lookup_ts')
stout_tractography.connect( pbx2, 'way_total', datasink, 'waytotal.txt')


#datasink.inputs.base_directory = put_cleaned_up_results_here
#datasink.inputs.substitutions = [('_subject_id_', ''),
#                                 ('_seed_..bigdata2..NIPYPE_WD..SANCHEZ_WD..', '')]

if __name__ == '__main__':
    stout_tractography.write_graph(format='eps')
    #stout_tractography.export_graph()
    # stout_tractography.run(plugin='MultiProc', plugin_args={'n_procs': 28})
    #dwiproc.run(plugin='PBS', plugin_args = dict(qsub_args='-k oe -q batch ') )
    stout_tractography.run(plugin='PBS', plugin_args=dict(qsub_args='-e trq_logs -o trq_logs -q batch '))

"""


###WORKINGDATAPATH

# $statement  = " convertwarp ";
# $statement .= " --ref=$WORKINGDATAPATH/MNI152_T1_1mm_brain.nii.gz ";
# $statement .= " --premat=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc.mat ";
# $statement .= " --warp1=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_warp_MNI_warpfield.nii.gz ";
# $statement .= " --out=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc_warp_MNI_warpfield.nii.gz ";
# print "$statement \n";

# $statement  = " convertwarp ";
# $statement .= " --ref=$WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain_mask.nii.gz ";
# $statement .= " --warp1=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/MNI_warp_struc_warpfield.nii.gz ";
# $statement .= " --postmat=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_12dof_nodif.mat ";
# $statement .= " --out=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/MNI_warp_struc_12dof_nodif_warpfield.nii.gz ";
# print "$statement \n";

# print "$statement \n";
"""